In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

import pid
from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./plots_9_26_23/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/"

onbeamfile = filedir + "data-reprod-C/NuMIMajorityRun1OnBeam_trk.df"
offbeamfile = filedir + "data-reprod-C/NuMIMajorityRun1OffBeam_trk.df"
mcfile = filedir + "mc/nu_trk_old.df"

mcfile = filedir + "mc/nu_fitwvfkstunegain_trk.df"
mcfile = filedir + "mc/mcnuphase2_trk_6000.df"

In [ ]:
mc = data.mc_dataset(mcfile, "trk", syst_weights=False)
mc.df

In [ ]:
onbeam = data.onbeam_dataset(onbeamfile, "trk")
onbeam.df

In [ ]:
offbeam = data.offbeam_dataset(offbeamfile, "trk")
offbeam.df

In [ ]:
POTSTR = "$%.2f \\times 10^{18}$ POT" % (onbeam.POT/1e18)

In [ ]:
importlib.reload(pid)

In [ ]:
def recalo(f, trkdf, isMC, betaf, chi2name="nom", gain=1, A=0.93, dqdxname="dqdx"):
    trkhitdf = pd.read_hdf(f, "hit")

    trkhitdf = trkhitdf[trkhitdf.rr < 30].copy()
    
    dir_x = trkdf.pfp.trk.dir.x
    dir_x.name = "dir_x"
    
    cryo = trkdf.pfp.trk.producer
    cryo.name = "cryostat"
    
    if trkhitdf.columns.nlevels > 1:
        trkhitdf = panda_helpers.multicol_add(trkhitdf, dir_x)
        trkhitdf = panda_helpers.multicol_add(trkhitdf, cryo)
    else:
        trkhitdf = trkhitdf.join(dir_x)
        trkhitdf = trkhitdf.join(cryo)

    trkhitdf.dir_x = trkhitdf.dir_x.fillna(False).astype(float)
    
    trkhitdf["itpc"] = trkhitdf.cryostat*2 + trkhitdf.tpc // 2
    
    trkhitdf["tpcEE"] = trkhitdf.itpc == 0
    trkhitdf["tpcEW"] = trkhitdf.itpc == 1
    trkhitdf["tpcWE"] = trkhitdf.itpc == 2
    trkhitdf["tpcWW"] = trkhitdf.itpc == 3
    

    # lifetime correction in MC
    if not isMC:
        CAL = 1
    else:
        trkhitdf["thit"] = (trkhitdf.t - 850)*0.4
        CAL = np.exp(trkhitdf.thit/3e3)

    trkhitdf["phi"] = np.arccos(np.abs(trkhitdf.dir_x))*180/np.pi
    trkhitdf["beta"] = betaf(trkhitdf.phi)
    
    trkhitdedx = pid.dedxdf(gain=gain*CAL, A=A, dqdxname=dqdxname, RR_QE=-1)(trkhitdf)
    
    chi2p = pid.hchi2p(trkhitdedx)
    chi2p.name = ("pfp", "trk", "chi2p_%s" % chi2name)
    if chi2p.name in trkdf.columns:
        del trkdf[chi2p.name]
    trkdf = panda_helpers.multicol_add(trkdf, chi2p)

    chi2u = pid.hchi2u(trkhitdedx)
    chi2u.name = ("pfp", "trk", "chi2u_%s" % chi2name)
    if chi2u.name in trkdf.columns:
        del trkdf[chi2u.name]
    trkdf = panda_helpers.multicol_add(trkdf, chi2u)
    
    return trkdf

In [ ]:
# angular beta

# Measurement 1
def ang_betaf(phi):
    return 1.71680806e-05*phi**2 -2.68462874e-03*phi + 3.07268501e-01

GAIN = 75.387
ALPHA = 0.9071

# Measurement 2
def ang_betaf(phi):
    return 2.01812433e-05*phi**2 -3.13322811e-03*phi + 3.28764217e-01

GAIN = 74.6826
ALPHA = 0.90223

# Measurement 3: EMB
def ang_betaf(phi, beta90=0.203, R=1.25):
    phi = phi * np.pi / 180 # deg to radian
    return beta90 / np.sqrt(np.sin(phi)**2 + np.cos(phi)**2/R**2)
GAIN = 75.1
ALPHA = 0.906

GAIN_GAIN_ONLY_FIT = 78.97 # GAIN ONLY FIT

# flat beta
def flat_betaf(phi):
    return 0.212

# MC Gain
MC_GAIN = 78.1 # MCNuPhase2

In [ ]:
1/0.01265

In [ ]:
1/0.01285

In [ ]:
# nominal chi2
onbeam.df = recalo(onbeamfile, onbeam.df, False, flat_betaf, gain=GAIN_GAIN_ONLY_FIT)

In [ ]:
# angular chi2
onbeam.df = recalo(onbeamfile, onbeam.df, False, ang_betaf, chi2name="ang", gain=GAIN, A=ALPHA)

In [ ]:
# nominal chi2
offbeam.df = recalo(offbeamfile, offbeam.df, False, flat_betaf, gain=GAIN_GAIN_ONLY_FIT)

In [ ]:
# angular chi2
offbeam.df = recalo(offbeamfile, offbeam.df, False, ang_betaf, chi2name="ang", gain=GAIN, A=ALPHA)

In [ ]:
# nominal chi2, MC
mc.df = recalo(mcfile, mc.df, True, flat_betaf, gain=MC_GAIN)

In [ ]:
# nominal chi2, MC
mc.df = recalo(mcfile, mc.df, True, flat_betaf, gain=1/0.01265, chi2name="altgain")

In [ ]:
mc.df[("pfp", "trk", "chi2u_ang", "", "", "")] = mc.df.pfp.trk.chi2u_nom
mc.df[("pfp", "trk", "chi2p_ang", "", "", "")] = mc.df.pfp.trk.chi2p_nom

In [ ]:
onbeam.df[("pfp", "trk", "chi2u_altgain", "", "", "")] = onbeam.df.pfp.trk.chi2u_ang
onbeam.df[("pfp", "trk", "chi2p_altgain", "", "", "")] = onbeam.df.pfp.trk.chi2p_ang

In [ ]:
offbeam.df[("pfp", "trk", "chi2u_altgain", "", "", "")] = offbeam.df.pfp.trk.chi2u_ang
offbeam.df[("pfp", "trk", "chi2p_altgain", "", "", "")] = offbeam.df.pfp.trk.chi2p_ang

In [ ]:
@var.VAR
def first(v):
    nlevel = v.index.nlevels
    return v.groupby(level=list(range(nlevel-1))).first()

In [ ]:
@var.VAR
def trk_contained(df):
    return TrkInFV(df.pfp.trk.end)

In [ ]:
bins = np.linspace(-1, 1, 21)
v = first@var.crlongtrkdiry@var.slc
c = first@cut.fiducial@var.slc
categories = map(lambda f: first@f@var.slc, cut.mc_nu_cosmic_categories)

hnom, hsyst = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, syst=None, wgtf=first,
                        categories=categories)
hon, _ = hist.makehist(v, onbeam, cut=c, bins=bins, cvweight=False, syst=None, wgtf=first)
hoff, _ = hist.makehist(v, offbeam, cut=c, bins=bins, cvweight=False, livetime=onbeam.livetime, syst=None, wgtf=first)

hmc, hfill = hist.plotmc(plt, hnom)
_, _, fh = hmc
hd = hist.plotdata(plt, hon-hoff)

legend1 = plt.legend(handles=[hd], labels=["Data"], loc="lower right", frameon=False)
plt.legend(handles=fh, labels=["$\\nu$", "Cosmic"], title="Monte Carlo",
                    loc="upper right")
plt.gca().add_artist(legend1)
    
plt.tight_layout()

plt.title("Prescaled ICARUS Data")

plt.xlabel("CRLongTrkDirY")
plt.ylabel("Slices / %s" % POTSTR)
plt.tight_layout()

if dosave: plt.savefig(savedir + "crongtrkdiry.pdf")

In [ ]:
chi2p_bins = np.linspace(0, 400, 41)
chi2u_bins = np.linspace(0, 80, 41)

basecut = cut.fiducial@var.slc & (var.TRKVAR.len > 5)

@var.VAR
def ntrk(df):
    return df.groupby(level=[0,1,2]).size()

cuts = [
    basecut,
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3),
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (var.TRKVAR.len > 100),
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & trk_contained
]

titles = [
    "Prescaled: Fiducial Tracks",
    "Prescaled: Fiducial, Cos. Rej. Tracks",
    "Prescaled: Fiducial, Cos. Rej., Long Tracks",
    "Prescaled: Fiducial, Cos. Rej., Contained Tracks"
]

labels = ["fid", "fid_cosrej", "fid_cosrej_long", "fid_cosrej_contained"]


PLT_AREANORM_ALL = True
PLT_AREANORM_CAT = True

In [ ]:
v = var.trk.chi2p_nom

for i, c in enumerate(cuts):
    if PLT_AREANORM_ALL:
        plt.figure(2*i)
        
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2p_bins, areanorm=True, syst=None)
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label="MC")
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")

        plt.xlabel("ArgoNeuT Recombination: $\\chi^2_p$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])
        plt.legend()
        plt.tight_layout()
                
        if dosave: plt.savefig(savedir + "chi2p_areanorm_%s.pdf" % labels[i])

    if PLT_AREANORM_CAT:
        plt.figure(2*i+1, figsize=(6.4*1.25, 4.8))
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2p_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label=cut.trk_mc_labels)
        
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")
        
        handles, labels = plt.gca().get_legend_handles_labels()
        legend1 = plt.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
        plt.legend(handles=handles[:-1], labels=labels[:-1],
            loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
        plt.gca().add_artist(legend1)
        
        plt.tight_layout()

        plt.xlabel("ArgoNeuT Recombination: $\\chi^2_p$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])

        
        plt.tight_layout()
        if dosave: plt.savefig(savedir + "chi2p_areanorm_categories_%s.pdf" % labels[i])            

In [ ]:
v = var.trk.chi2p_ang

for i, c in enumerate(cuts):
    if PLT_AREANORM_ALL:
        plt.figure(2*i)
        
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2p_bins, areanorm=True, syst=None)
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label="MC")
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")

        plt.xlabel("EMB Recombination: $\\chi^2_p$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])
        plt.legend()
        plt.tight_layout()
                
        if dosave: plt.savefig(savedir + "chi2p_ang_areanorm_%s.pdf" % labels[i])

    if PLT_AREANORM_CAT:
        plt.figure(2*i+1, figsize=(6.4*1.25, 4.8))
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2p_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label=cut.trk_mc_labels)
        
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")
        
        handles, labels = plt.gca().get_legend_handles_labels()
        legend1 = plt.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
        plt.legend(handles=handles[:-1], labels=labels[:-1],
            loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
        plt.gca().add_artist(legend1)
        
        plt.tight_layout()

        plt.xlabel("EMB Recombination: $\\chi^2_p$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])

        
        plt.tight_layout()
        if dosave: plt.savefig(savedir + "chi2p_ang_areanorm_categories_%s.pdf" % labels[i])            

In [ ]:
v = var.trk.chi2u_nom

for i, c in enumerate(cuts):
    if PLT_AREANORM_ALL:
        plt.figure(2*i)
        
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None)
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label="MC")
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")

        plt.xlabel("ArgoNeuT Recombination: $\\chi^2_\\mu$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])
        plt.legend()
        plt.tight_layout()
                
        if dosave: plt.savefig(savedir + "chi2u_areanorm_%s.pdf" % labels[i])

    if PLT_AREANORM_CAT:
        plt.figure(2*i+1, figsize=(6.4*1.25, 4.8))
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label=cut.trk_mc_labels)
        
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")
        
        handles, labels = plt.gca().get_legend_handles_labels()
        legend1 = plt.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
        plt.legend(handles=handles[:-1], labels=labels[:-1],
            loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
        plt.gca().add_artist(legend1)
        
        plt.tight_layout()

        plt.xlabel("ArgoNeuT Recombination: $\\chi^2_\\mu$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])

        
        plt.tight_layout()
        if dosave: plt.savefig(savedir + "chi2u_areanorm_categories_%s.pdf" % labels[i])            

In [ ]:
v = var.trk.chi2u_ang

for i, c in enumerate(cuts):
    if PLT_AREANORM_ALL:
        plt.figure(2*i)
        
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None)
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label="MC")
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")

        plt.xlabel("EMB Recombination: $\\chi^2_\\mu$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])
        plt.legend()
        plt.tight_layout()
                
        if dosave: plt.savefig(savedir + "chi2u_ang_areanorm_%s.pdf" % labels[i])

    if PLT_AREANORM_CAT:
        plt.figure(2*i+1, figsize=(6.4*1.25, 4.8))
        hnom,_ = hist.makehist(v, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
        hon,_ = hist.makehist(v, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
        hoff,_ = hist.makehist(v, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                               livetime=onbeam.livetime, syst=None)
        
        hmc, hfill = hist.plotmc(plt, hnom, label=cut.trk_mc_labels)
        
        hd = hist.plotdata(plt, (hon-hoff).to_area(), label="Data")
        
        handles, labels = plt.gca().get_legend_handles_labels()
        legend1 = plt.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
        plt.legend(handles=handles[:-1], labels=labels[:-1],
            loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
        plt.gca().add_artist(legend1)
        
        plt.tight_layout()

        plt.xlabel("EMB Recombination: $\\chi^2_\\mu$")
        plt.ylabel("Area Normalized")
        plt.title(titles[i])

        
        plt.tight_layout()
        if dosave: plt.savefig(savedir + "chi2u_ang_areanorm_categories_%s.pdf" % labels[i])            

In [ ]:
c = cuts[-1]
vl = var.trk.chi2u_nom
vr = var.trk.chi2u_ang


hnomul,_ = hist.makehist(vl, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
honul,_ = hist.makehist(vl, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
hofful,_ = hist.makehist(vl, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

hnomur = hnomul
honur,_ = hist.makehist(vr, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
hoffur,_ = hist.makehist(vr, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

In [ ]:
fig, (al,ar) = plt.subplots(1, 2, width_ratios=[1, 1], figsize=(6.4*1.75, 4.8), sharey=True)
fig.subplots_adjust(wspace=0.025)

hmc, hfill = hist.plotmc(al, hnomul, label=cut.trk_mc_labels)
hd = hist.plotdata(al, (honul-hofful).to_area(), label="Data (ON Beam - OFF)")


hmc, hfill = hist.plotmc(ar, hnomur, label=cut.trk_mc_labels)
hd = hist.plotdata(ar, (honur-hoffur).to_area(), label="Data (ON Beam - OFF)")

handles, labels = ar.get_legend_handles_labels()
legend1 = ar.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
ar.legend(handles=handles[:-1], labels=labels[:-1],
    loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
ar.add_artist(legend1)

al.set_xlabel("ArgoNeuT Mod. Box Recombination: $\\chi^2_\\mu$")
al.set_ylabel("Area Normalized")

ar.set_xlabel("Ellipsoid Mod. Box Recombination: $\\chi^2_\\mu$")
fig.suptitle("Contained, Cosmic Rejected Tracks", y=0.95, fontsize=20)

if dosave: plt.savefig(savedir + "chi2u_angnom_areanorm_categories.pdf", bbox_inches="tight")            

In [ ]:
c = cuts[-1]
vl = var.trk.chi2p_nom
vr = var.trk.chi2p_ang


hnompl,_ = hist.makehist(vl, mc, cut=c, bins=chi2p_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
honpl,_ = hist.makehist(vl, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
hoffpl,_ = hist.makehist(vl, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

hnompr = hnompl
honpr,_ = hist.makehist(vr, onbeam, cut=c, bins=chi2p_bins, cvweight=False, syst=None)
hoffpr,_ = hist.makehist(vr, offbeam, cut=c, bins=chi2p_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

In [ ]:
fig, (al,ar) = plt.subplots(1, 2, width_ratios=[1, 1], figsize=(6.4*1.75, 4.8), sharey=True)
fig.subplots_adjust(wspace=0.025)

hmc, hfill = hist.plotmc(al, hnompl, label=cut.trk_mc_labels)
hd = hist.plotdata(al, (honpl-hoffpl).to_area(), label="Data (ON Beam - OFF)")


hmc, hfill = hist.plotmc(ar, hnompr, label=cut.trk_mc_labels)
hd = hist.plotdata(ar, (honpr-hoffpr).to_area(), label="Data (ON Beam - OFF)")

handles, labels = ar.get_legend_handles_labels()
legend1 = ar.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
ar.legend(handles=handles[:-1], labels=labels[:-1],
    loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
ar.add_artist(legend1)

al.set_xlabel("ArgoNeuT Mod. Box Recombination: $\\chi^2_\\mu$")
al.set_ylabel("Area Normalized")

ar.set_xlabel("Ellipsoid Mod. Box Recombination: $\\chi^2_\\mu$")
# fig.suptitle("Contained, Cosmic Rejected Tracks", y=0.95, fontsize=20)

if dosave: plt.savefig(savedir + "chi2p_angnom_areanorm_categories.pdf", bbox_inches="tight")            

In [ ]:
c = cuts[-1]
vl = var.trk.chi2u_altgain
vr = var.trk.chi2u_ang


hnomal,_ = hist.makehist(vl, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
honal,_ = hist.makehist(vl, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
hoffal,_ = hist.makehist(vl, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

hnomal,_ = hist.makehist(vr, mc, cut=c, bins=chi2u_bins, areanorm=True, syst=None, 
                               categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
honar,_ = hist.makehist(vr, onbeam, cut=c, bins=chi2u_bins, cvweight=False, syst=None)
hoffar,_ = hist.makehist(vr, offbeam, cut=c, bins=chi2u_bins, cvweight=False, 
                       livetime=onbeam.livetime, syst=None)

In [ ]:
fig, (al,ar) = plt.subplots(1, 2, width_ratios=[1, 1], figsize=(6.4*2.25, 4.8), sharey=True)
fig.subplots_adjust(wspace=0.025)

hmc, hfill = hist.plotmc(al, hnomal, label=cut.trk_mc_labels)
hd = hist.plotdata(al, (honal-hoffal).to_area(), label="Data (ON Beam - OFF)")


hmc, hfill = hist.plotmc(ar, hnomar, label=cut.trk_mc_labels)
hd = hist.plotdata(ar, (honar-hoffar).to_area(), label="Data (ON Beam - OFF)")

handles, labels = ar.get_legend_handles_labels()
legend1 = ar.legend(handles=[handles[-1]], labels=[labels[-1]], loc="upper right", frameon=False)
ar.legend(handles=handles[:-1], labels=labels[:-1],
    loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))
ar.add_artist(legend1)

al.set_xlabel("High Gain: $\\chi^2_\\mu$")
al.set_ylabel("Area Normalized")

ar.set_xlabel("Low Gain: $\\chi^2_\\mu$")
fig.suptitle("Contained, Cosmic Rejected Tracks", y=0.95, fontsize=20)

if dosave: plt.savefig(savedir + "chi2u_angnom_altgain_areanorm_categories.pdf", bbox_inches="tight")            

In [ ]:
fig, (al,ar) = plt.subplots(1, 2, width_ratios=[1, 1], figsize=(6.4*2.25, 4.8))
# fig.subplots_adjust(wspace=0.025)

hmc, hfill = hist.plotmc(al, hnomul, label=cut.trk_mc_labels)
hmc, hfill = hist.plotmc(ar, hnompl, label=cut.trk_mc_labels)

handles, labels = ar.get_legend_handles_labels()
ar.legend(handles=handles, labels=labels,
    loc="upper left", title="MC Categories", bbox_to_anchor=(1, 1))

al.set_xlabel("$\\chi^2_\\mu$")
al.set_ylabel("Area Normalized")

ar.set_xlabel("$\\chi^2_p$")
ar.set_ylabel("Area Normalized")
fig.suptitle("Contained, Cosmic Rejected Tracks", y=0.95, fontsize=20)

if dosave: plt.savefig(savedir + "chi2u_mconly_areanorm_categories.pdf", bbox_inches="tight")            

In [ ]:
# proton-like selection
protonlike = basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & trk_contained &\
    (var.trk.chi2u_ang > 35) & (var.trk.chi2p_ang < 50)

v = var.pfp.trackScore

bins = np.linspace(0, 1, 21)

In [ ]:
plt.figure(4*i+3, figsize=(6.4*1.25, 4.8))
hmc = hist.plotmc(plt, v, mc, cut=protonlike, bins=bins, areanorm=True,
                categories=map(lambda f: f@var.trk, cut.trk_mc_categories))

hd = hist.plotdata(plt, v, onbeam, offbeam, cut=protonlike, bins=bins, areanorm=True)
hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)

plt.xlabel("Pandora Track Score")
plt.ylabel("Area Normalized")
plt.title("Contained Proton-Like Particles")

plt.tight_layout()
if dosave: plt.savefig(savedir + "trackscore_protonlike.pdf")


In [ ]:
# muon-like selection
muonlike = basecut & (var.crlongtrkdiry@var.slc > -0.6) & (ntrk >= 2) & trk_contained &\
    (var.trk.chi2u_ang < 10) & (var.trk.chi2p_ang > 80)  & (var.trk.len > 150)

In [ ]:
plt.figure(4*i+3, figsize=(6.4*1.25, 4.8))
hmc = hist.plotmc(plt, v, mc, cut=muonlike, bins=bins, areanorm=True,
                categories=map(lambda f: f@var.trk, cut.trk_mc_categories))

hd = hist.plotdata(plt, v, onbeam, offbeam, cut=muonlike, bins=bins, areanorm=True)
hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)

plt.xlabel("Pandora Track Score")
plt.ylabel("Area Normalized")
plt.title("Contained Muon-Like Particles")

plt.tight_layout()
if dosave: plt.savefig(savedir + "trackscore_muonlike.pdf")
